In [1]:
require 'nngraph';

function createFullModel(B_model, C_model)
    local image_feat1 = nn.Identity()();
    local image_feat2 = nn.Identity()();
    local image_feat3 = nn.Identity()();
    local question = nn.Identity()();

    local confidence = B_model({question, image_feat3});
    local scores = C_model({image_feat1, image_feat2, question, confidence});
    
    nngraph.annotateNodes();
    return nn.gModule({image_feat1, image_feat2, image_feat3, question}, {scores});
end

-- function to get an example for training C
function getCtrainExample(set, labels)

    -- randomly select two images from different classes
    torch.manualSeed(3489208)
    local y = torch.randperm((#set)[1])

    local im_feat = {}
    im_feat[1] = set[y[1]]
    im_feat[2] = set[y[2]]

    -- randomly select one of those to be given to model B
    local label = torch.bernoulli() + 1
    im_feat[3] = im_feat[label]:clone()

    -- randomly select one of the images to ask about its class
    local ques = torch.Tensor(42):fill(0)
    ques[labels[y[torch.bernoulli() + 1]]] = 1

    -- target is probability of label = 2
    local target = label - 1

    -- Testing
    -- outfile:write(im1_Path)
    -- outfile:write(im2_Path)
    -- outfile:write(y[1], y[2])
    -- outfile:write(label)
    -- outfile:write(ques[y[1]], ques[y[2]])
    
    local input = {im_feat[1], im_feat[2], im_feat[3], ques}
    
    return {input, target}
end

function do_weight_decay(model, wd)
    lin_modules = model:findModules('nn.Linear');
    for i = 1,#lin_modules do
        m = torch.mul(lin_modules[i].weight, wd);
        lin_modules[i].gradWeight = lin_modules[i].gradWeight + m;
    end
end

function get_pred_err(prob, target)

    local pred = 2
    if(prob[1] < 0.5) then
        pred = 1
    end

    local pred_err = 0
    if pred ~= target+1 then
        pred_err = 1
    end

    return pred_err
end

-- Use a typical generic gradient update function
function accumulate(model, input, target, criterion, eval_criterion, batch_size, wd)
    local prob = model:forward(input)
    local loss = criterion:forward(prob, torch.Tensor{target})
    local gradCriterion = criterion:backward(prob, torch.Tensor{target})
    model:backward(input, gradCriterion, 1/batch_size)
    do_weight_decay(model, wd)
    local pred_err = get_pred_err(prob, target)
    -- print('prob = ', prob)
    return loss, pred_err
end


-- function to evalaute the model
function evalPerf(model, criterion, set, labels, iter)

    outfile = io.open("train_C.out", "a")
    outfile:write('Testing... ')
    outfile:close()

    local test_loss = 0
    local test_pred_err = 0

    -- FOR DEBUGGING only
    -- set the random seed so that same batch is chosen always. Make sure error goes down
    -- torch.manualSeed(3489208)

    for j = 1, iter do
        example = getCtrainExample(set, labels)
        input = example[1]
        target = example[2]
        local prob = model:forward({input[1], input[2], input[3], input[4]})
        local samp_loss = criterion:forward(prob, torch.Tensor{target})
        local pred_err = get_pred_err(prob, target)
        test_pred_err = test_pred_err + pred_err
        test_loss = test_loss + samp_loss
    end

    outfile = io.open("train_C.out", "a")
    outfile:write('average test_loss = ', test_loss/iter, ', ')
    outfile:write('average test_pred_err = ', test_pred_err/iter, '\n')
    outfile:close()
end

In [2]:
-- Laod the original model and creat BC model
B_model = torch.load('B_model_nn.t7')
C_model = torch.load('C_model__1500.t7')
BC_model = createFullModel(B_model, C_model)
BC_model:double()
BC_model:evaluate()

In [3]:
-- read preprocessed feature vectors and labels
feat_vecs = torch.load('feat_vecs.t7')
labels = torch.load('labels.t7')

-- generate trainset and testset
train_perc = 0.80 -- percentage of images in the train set
trainset_size = torch.round((#feat_vecs)[1] * train_perc)
trainset = feat_vecs[{{1, trainset_size}}]
train_labels = labels[{{1, trainset_size}}]
testset = feat_vecs[{{trainset_size+1, (#feat_vecs)[1]}}]
test_labels = labels[{{trainset_size+1, (#feat_vecs)[1]}}]

In [4]:
crit = nn.BCECriterion()
eval_crit = crit
lr = 2
attr_lr = 0.1
batch_size = 1
max_train_iter = 10000
test_interval = 50
test_iter = 1000
lr_stepsize = 100
gamma = 1
attr_gamma = 1
wd = 0
snapshot_interval = 100
snapshot_prefix = './'
snapshot = false

In [5]:
example = getCtrainExample(trainset, train_labels)
input = example[1]
target = example[2]
confidence = torch.Tensor{0.8}

In [6]:
print(torch.max(input[1]))
print(torch.max(input[2]))
print(torch.max(input[3]))
print(torch.max(input[4]))

5.6793746948242	
5.6605820655823	
5.6793746948242	
1	


In [7]:
C_model:forward({input[1], input[2], input[4], confidence})

1e-16 *
 2.4476
[torch.DoubleTensor of size 1]



In [ ]:
-- Start training
outfile = io.open('train_C.out', 'a')
outfile:write('Training with snapshotting ')
if snapshot then
    outfile:write('enabled... \n')
else
    outfile:write('disabled... \n')
end
outfile:close()

for i = 1, max_train_iter do

    -- initial testing
    if i == 1 then
        evalPerf(BC_model, eval_crit, testset, test_labels, test_iter)
    end

    BC_model:zeroGradParameters()
    local batch_loss = 0

    -- FOR DEBUGGING only
    -- set the random seed so that same batch is chosen always. Make sure error goes down
    -- torch.manualSeed(214325)

    local train_pred_err = 0
    for j = 1, batch_size do
        example = getCtrainExample(trainset, train_labels)
        input = example[1]
        target = example[2]
        local loss = 0
        local pred_err = 0
        loss, pred_err = accumulate(BC_model, {input[1], input[2], input[3], input[4]}, target, crit, eval_crit, batch_size, wd)
        batch_loss = batch_loss + loss
        train_pred_err = train_pred_err + pred_err;
        -- print(C_model.modules[9].output)
        -- print('loss =', loss)
        -- print(target)
    end

    -- update parameters for only a few layers in C
    -- C_model.modules[2]:updateParameters(attr_lr)
    -- C_model.modules[6]:updateParameters(attr_lr)
    -- C_model.modules[10]:updateParameters(lr)

    outfile = io.open("train_C.out", "a")
    outfile:write('Iteration no. ', i, ', lr = ', lr, ', attr_lr = ', attr_lr, ', batch_loss = ', batch_loss/batch_size, ', train_err = ', train_pred_err/batch_size, '\n')
    outfile:close()

    if i % test_interval == 0 then
        evalPerf(BC_model, eval_crit, testset, test_labels, test_iter)        
    end

    if i % lr_stepsize == 0 then
        lr = lr * gamma;
        attr_lr = attr_lr * attr_gamma;
    end

    if snapshot and (i % snapshot_interval == 0) then

        outfile = io.open("train_C.out", "a")

        outfile:write('Snapshotting C_model... ')
        snapshot_filename_C = snapshot_prefix .. 'C_model__' .. tostring(i) .. '.t7'
        C_model:clearState()
        torch.save(snapshot_filename_C, C_model)
        outfile:write('done\n')

        outfile:close()

    end

end